# **How to Use Colab Pro to Do Evaluations!**

**We suggest to use Colab pro's A100 GPU to do the evaluations**



**Important: When you run, make sure you choose runtime A100 GPU**

Git and install libraries

In [ ]:
!git clone https://github.com/The-FinAI/PIXIU --recursive
%cd PIXIU
!pip install -r requirements.txt
%cd /content/PIXIU/src/financial-evaluation
!pip install -e .[multilingual]
!pip install bert_score
!pip install vllm==0.2.7

Cloning into 'PIXIU'...
remote: Enumerating objects: 944, done.
remote: Counting objects: 100% (383/383), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 944 (delta 272), reused 285 (delta 217), pack-reused 561
Receiving objects: 100% (944/944), 50.01 MiB | 20.84 MiB/s, done.
Resolving deltas: 100% (515/515), done.
Submodule 'FinMem-LLM-StockTrading' (https://github.com/pipiku915/FinMem-LLM-StockTrading.git) registered for path 'FinMem-LLM-StockTrading'
Submodule 'src/financial-evaluation' (https://github.com/chancefocus/financial-evaluation.git) registered for path 'src/financial-evaluation'
Submodule 'src/metrics/BARTScore' (https://github.com/neulab/BARTScore.git) registered for path 'src/metrics/BARTScore'
Cloning into '/content/PIXIU/FinMem-LLM-StockTrading'...
remote: Enumerating objects: 325, done.        
remote: Counting objects: 100% (47/47), done.        
remote: Compressing objects: 100% (41/41), done.        
remote: Total 325 (delta 23), reused 14 (

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.1 MB/s eta 0:00:00


Follow the Readme page download BART checkpoint to src/metrics/BARTScore/

you can put bart_score.pth*bart_score.pth* file under your Google drive, connect drive to upload *bart_score.pth* to colab PIXIU folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "/content/drive/My Drive/bart_score.pth" "/content/PIXIU/src/metrics/BARTScore/bart_score.pth"

Mounted at /content/drive


When doing *bart score* related evaluation, you need to modify the code manually.

Click *folder* on the left.

/content/PIXIU/src/tasks/flare.py

row 374 and 513

bart_scorer.load(path="src/metrics/BARTScore/bart_score.pth")

change to

bart_scorer.load(path="/content/PIXIU/src/metrics/BARTScore/bart_score.pth")

In [ ]:
%cd /content/PIXIU/src

/content/PIXIU/src


Modify the scrpits when you evaluate.

e.g.

pretrained=ChanceFocus/finma-7b-full,tokenizer=ChanceFocus/finma-7b-full

pretrained=tiiuae/falcon-7b,tokenizer=tiiuae/falcon-7b

you can find all the models on huggingface.

Sentiment Analysis:

In [ ]:
import os

os.environ['PYTHONPATH'] += ":/content/PIXIU/src/metrics/BARTScore/"
!echo $PYTHONPATH
!python eval.py \
    --model "hf-causal-vllm" \
    --model_args "use_accelerate=True,pretrained=TheFinAI/finma-7b-full,tokenizer=TheFinAI/finma-7b-full,use_fast=False,max_gen_toks=20,dtype=float16" \
    --tasks "flare_tsa" \
    --batch_size 30000

/env/python:/content/PIXIU/src/metrics/BARTScore/:/content/PIXIU/src/metrics/BARTScore/:/content/PIXIU/src/metrics/BARTScore/:/content/PIXIU/src/metrics/BARTScore/
2024-04-13 14:29:28.109222: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 14:29:28.109280: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 14:29:28.111264: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 14:29:29.332103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Pac

Classification:

In [ ]:
!python eval.py \
    --model "hf-causal-vllm" \
    --model_args "use_accelerate=True,pretrained=TheFinAI/finma-7b-full,tokenizer=TheFinAI/finma-7b-full,use_fast=False,max_gen_toks=20,dtype=float16" \
    --tasks "flare_finarg_ecc_auc,flare_finarg_ecc_arc,flare_cfa,flare_multifin_en,flare_ma,flare_mlesg" \
    --batch_size 30000

2024-04-13 13:51:58.034434: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 13:51:58.034489: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 13:51:58.036435: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 13:51:59.244673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Selected Tasks: ['flare_cfa', 'flare_finarg_ecc_arc', 'flare_finarg_ecc_auc', 'flare_ma', 'flare_mlesg', 'flare_multifin_en']
INFO

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



, its strategy to establish India as a major green hydrogen production hub, with plans to reach 5 million tonnes of production, along with over $2 billion in incentives for related infrastructure and production.', 'choices': ['Access to Communications', 'Biodiversity & Land Use', 'Packaging Material & Waste', 'Financing Environmental Impact', 'Carbon Emissions', 'Human Capital Development', 'Ownership & Control', 'Community Relations', 'Responsible Investment', 'Opportunities in Renewable Energy', 'Consumer Financial Protection', 'Accounting', 'Business Ethics', 'Opportunities in Clean Tech', 'Toxic Emissions & Waste', 'Product Carbon Footprint', 'Opportunities in Green Building', 'Climate Change Vulnerability', 'Pay', 'Water Stress', 'Supply Chain Labor Standards', 'Chemical Safety', 'Board', 'Opportunities in Nutrition & Health', 'Access to Health Care', 'Electronic Waste', 'Access to Finance', 'Raw Material Sourcing', 'Health & Demographic Risk', 'Labor Management', 'Controversial S

Knowledge Extraction:

In [ ]:
!python eval.py \
    --model "hf-causal-vllm" \
    --model_args "use_accelerate=True,pretrained=TheFinAI/finma-7b-full,tokenizer=TheFinAI/finma-7b-full,max_gen_toks=128,dtype=float16" \
    --tasks "flare_finred,flare_causal20_sc,flare_cd" \
    --batch_size 30000

2024-04-13 14:01:35.198824: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 14:01:35.198874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 14:01:35.200841: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 14:01:36.393299: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Selected Tasks: ['flare_causal20_sc', 'flare_cd', 'flare_finred']
INFO 04-13 14:01:41 llm_engine.py:70] Initializing an LLM engine

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



doc: {'id': 'finred335', 'query': "Given the following sentence, identify the head, tail, and relation of each triplet present in the sentence. The relations you should be looking for are 'legal_form', 'publisher', 'owner_of', 'employer', 'manufacturer', 'position_held', 'chairperson', 'industry', 'business_division', 'creator', 'original_broadcaster', 'chief_executive_officer', 'location_of_formation', 'operator', 'owned_by', 'founded_by', 'parent_organization', 'member_of', 'product_or_material_produced', 'brand', 'headquarters_location', 'director_/_manager', 'distribution_format', 'distributed_by', 'platform', 'currency', 'subsidiary', 'stock_exchange', and 'developer'. If a relation exists between two entities, provide your answer in the format 'head ; tail ; rel'. If there are multiple triplets in a sentence, provide each one on a new line.\nText: Movers: Glencore PLC GLEN, +16.95% GLCNF, +16.45% surged 17% in London after analysts at Citigroup suggested the mining group should g

Number Understanding:

In [ ]:
!python eval.py \
    --model "hf-causal-vllm" \
    --model_args "use_accelerate=True,pretrained=TheFinAI/finma-7b-full,tokenizer=TheFinAI/finma-7b-full,max_gen_toks=128,dtype=float16" \
    --tasks "flare_tatqa,flare_fnxl,flare_fsrl" \
    --batch_size 30000

2024-04-13 14:15:35.911447: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 14:15:35.911504: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 14:15:35.913155: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 14:15:37.132343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Selected Tasks: ['flare_fnxl', 'flare_fsrl', 'flare_tatqa']
INFO 04-13 14:15:42 llm_engine.py:70] Initializing an LLM engine with 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



tion based on the context.\nContext: |($ in millions)||||\n|For the year ended December 31:|2019|2018|Yr.-to-Yr. Percent Change|\n|Interest expense|$1,344|$723|85.9%|\n|Non-operating adjustment||||\n|Acquisition-related charges|(228)|—|NM|\n|Operating (non-GAAP) interest expense|$1,116|$723|54.4|\nInterest Expense\nNM-not meaningful\nInterest expense increased $621 million compared to 2018. Interest expense is presented in cost of financing in the Consolidated Income Statement only if the related external borrowings are to support the Global Financing external business. Overall interest expense (excluding capitalized interest) in 2019 was $1,952 million, an increase of $473 million year to year, driven by a higher average debt balance and higher interest rates as we issued debt to finance the Red Hat acquisition.\nOperating (non-GAAP) interest expense increased $393 million compared to the prior-year period. It excludes the Red Hat pre-closing debt financing costs.\nQuestion: What was 

In [ ]:

!python eval.py \
    --model "hf-causal-vllm" \
    --model_args "use_accelerate=True,pretrained=TheFinAI/finma-7b-full,tokenizer=TheFinAI/finma-7b-full,max_gen_toks=128,dtype=float16" \
    --tasks "flare_convfinqa" \
    --batch_size 30000

2024-04-13 14:24:41.171464: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 14:24:41.171520: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 14:24:41.173194: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 14:24:42.387997: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Selected Tasks: ['flare_convfinqa']
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/huggingface

Text Summarization:

In [ ]:
!python eval.py \
    --model "hf-causal-vllm" \
    --model_args "use_accelerate=True,pretrained=TheFinAI/finma-7b-full,tokenizer=TheFinAI/finma-7b-full,max_gen_toks=128,dtype=float16" \
    --tasks "flare_ectsum,flare_edtsum" \
    --batch_size 30000

Credit Scoring:

In [ ]:
!python eval.py \
    --model "hf-causal-vllm" \
    --model_args "use_accelerate=True,pretrained=TheFinAI/finma-7b-full,tokenizer=TheFinAI/finma-7b-full,max_gen_toks=128,dtype=float16" \
    --tasks "flare_cra_lendingclub,flare_cra_ccf,flare_cra_ccfraud,flare_cra_polish,flare_cra_taiwan,flare_cra_portoseguro,flare_cra_travelinsurace" \
    --batch_size 30000

2024-02-15 07:49:25.046536: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 07:49:25.046585: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 07:49:25.047983: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-15 07:49:26.225990: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Selected Tasks: ['flare_cra_ccf', 'flare_cra_ccfraud', 'flare_cra_lendingclub', 'flare_cra_polish', 'flare_cra_portoseguro', 'flare_cra_taiwan', 'flare_cra_travelinsurace']
INFO 02-15 07:49:31 llm_engine.py:70] Initializing an LLM engine wit

Forecasting:

In [ ]:
!python eval.py \
    --model "hf-causal-vllm" \
    --model_args "use_accelerate=True,pretrained=TheFinAI/finma-7b-full,tokenizer=TheFinAI/finma-7b-full,max_gen_toks=128,dtype=float16" \
    --tasks "flare_sm_bigdata,flare_sm_acl,flare_sm_cikm" \
    --batch_size 30000